INSTALLING ALL THE DEPENDENCIES

In [ ]:
!sudo apt-get install tesseract-ocr-[hin]
!pip install pytesseract
!pip install pdf2image
!apt-get install poppler-utils
! apt install tesseract-ocr
! apt install libtesseract-dev

IMPORTING ALL THE DEPENDENCIES

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import pytesseract
import tqdm
import pandas as pd
import string
import os
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

CODE TO GET IMAGES OF THE PAGES FROM THE PDFs

In [ ]:
from pdf2image import convert_from_path

images = convert_from_path(r'/content/drive/MyDrive/Layout_Analysis_Data/Tesseract_dict/pdfs/Bhandaran Shabdawali - 08-06-23 New.pdf')

for i in tqdm.tqdm(range(len(images))):
	images[i].save('/content/drive/MyDrive/Layout_Analysis_Data/Tesseract_dict/pages3/bhandaran_page'+ str(i) +'.jpg', 'JPEG')


EXTRACT THE LIST OF ENGLISH WORD, HINDI MEANING AND HINDI DESCRIPTION FOR A PAGE

In [ ]:
def extract(img_path):
  img=cv2.imread(img_path)
  # getting the tesseract result for the page
  words=pytesseract.image_to_string(img,lang='hin+eng',config='--psm 6')
  word=words.split('\n')
  eng=''
  hin=''
  des=''
  data=[]
  for i in word:
    if len(i)>4 and i[0] not in string.punctuation and not i[0].isdigit():
      if ord(i[0])<150 and len(i.strip().split(' ')[0])>0:
        if len(hin)>1:
          # print([eng,hin,des])
          data.append([eng,hin,des])
        eng=''
        hin=''
        des=''
        i=i.strip().split()
        for j in i:
          if j in '-|:;.=':
            continue
          elif ord(j[0])<150:
            eng+=j+' '
          else:
            hin+=j+' '
      else:
        des+=i
  # print([eng,hin,des])
  if len(hin)>1:
    data.append([eng,hin,des])
  return data

DRIVER CODE WHERE YOU CAN PASS THE PDF AND GET A LIST OF ALL ENGLISH WORD, HINDI MEANING AND HINDI DESCRIPTION FOR THE PDF

In [ ]:
result=[]
path='/content/drive/MyDrive/Layout_Analysis_Data/Tesseract_dict/pages3'
for img in tqdm.tqdm(os.listdir(path)):
  try:
    result.extend(extract(path+'/'+img))
  except:
    pass
for i in result:
  print(i)

CODE TO GET THE CSV FILE FOR THE EXTRACTED OUTPUT

In [ ]:
import csv
fields=['English Word','Hindi Meaning','Hindi Description']
with open('pdf2.csv','w') as f:
  write=csv.writer(f)
  write.writerow(fields)
  write.writerows(result)